In [1]:
import os
from os.path import join
import pickle
import torch
import numpy as np
from pysdf import SDF
import trimesh
from manotorch.manolayer import ManoLayer
from tqdm import *

def convert_to_obj_frame(pc, obj_rot, obj_trans):
    pc = (obj_rot.T @ (pc - obj_trans).T).T
    return pc

def get_close(contact):
        return np.sum(np.abs(contact[np.abs(contact)<0.01]))

datapath = '/root/code/seqs/1205_data/'
obj_path = '/root/code/seqs/object/'
manolayer = ManoLayer(
    mano_assets_root='/root/code/CAMS/data/mano_assets/mano',
    side='right'
)

mano_knuckles_file = '/root/code/CAMS/data/preparation/mano_parts_1.pt' # MANO part file of knuckles
mano_parts_d = torch.load(mano_knuckles_file) # 1552面
verts_part_d = np.zeros(778, dtype=np.int32)
mano_faces = manolayer.get_mano_closed_faces()
for face_id in range(mano_faces.shape[0]):
    verts_part_d[mano_faces[face_id, 0]] = mano_parts_d[face_id]
    verts_part_d[mano_faces[face_id, 1]] = mano_parts_d[face_id]
    verts_part_d[mano_faces[face_id, 2]] = mano_parts_d[face_id]
verts_part_d = torch.from_numpy(verts_part_d)

tips = [15,3,6,12,9]

seqs = sorted(os.listdir(datapath))
grasp_id = []

    


/root/anaconda3/envs/mdm/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'pysdf'

In [15]:
grasp_dict = {}
seqs = sorted(os.listdir(datapath))
for seq in tqdm(seqs):
    seq_path = join(datapath,seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    if meta['hand_flag'] == 'right':
        hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
        hand_trans = hand_params[:,:3]
        hand_theta = hand_params[:,3:51]
        mano_beta = hand_params[:,51:]
        active_obj = meta['active_obj']
        obj_mesh_path = join(obj_path,active_obj,'simplified_scan_processed.obj')
        obj_mesh = trimesh.load(obj_mesh_path)
        obj_pose = torch.tensor(np.load(join(seq_path,active_obj+'_pose_trans.npy'))).float()
        obj_rot = obj_pose[:,:,:3]
        obj_trans = obj_pose[:,:,3]
        obj_sdf = SDF(obj_mesh.vertices,obj_mesh.faces)
        for i in range(len(hand_params)):
            thumb_flag = False
            finger_flag = False
            save_flag = False
            mano_output = manolayer(hand_theta[i].unsqueeze(0), mano_beta[i].unsqueeze(0))
            mano_verts = mano_output.verts[0] - mano_output.joints[0, 0] + hand_trans[i]
            mano_verts = convert_to_obj_frame(mano_verts.squeeze(),
                                                obj_rot[i],
                                                obj_trans[i])
            # hand_faces = manolayer.get_mano_closed_faces()
            hand_contact = obj_sdf(mano_verts)
            # print(hand_contact.shape)
            if get_close(hand_contact) > 0:
                for tip in tips:
                    tip_contact = hand_contact[verts_part_d == tip]
                    tip_close = get_close(tip_contact)
                    if tip == 15:
                        if tip_close > 0:
                            thumb_flag = True
                    else:
                        if tip_close > 0 :
                            finger_flag = True
                if thumb_flag and finger_flag:
                    grasp_dict[seq] = i
                    # print(seq,i)
                    break
            else:
                if i == len(hand_params) - 1:
                    print(seq,"failed.")
        # break
    else:
        continue

 13%|█▎        | 185/1378 [19:25<2:13:06,  6.69s/it]

0185 failed.


 25%|██▍       | 338/1378 [38:16<6:04:50, 21.05s/it]

0338 failed.


 25%|██▍       | 341/1378 [39:29<6:27:11, 22.40s/it]

...

 78%|███████▊  | 1072/1378 [2:42:51<37:27,  7.34s/it]  

1072 failed.


 87%|████████▋ | 1194/1378 [2:56:15<57:33, 18.77s/it]  

1194 failed.


100%|██████████| 1378/1378 [3:37:48<00:00,  9.48s/it]  


In [ ]:
# grasp_dict = {}
def get_close(contact):
        return np.sum(np.abs(contact[np.abs(contact)<0.03]))
fail_seqs = ['0185','0338','0341','0381','0484','0605','0756','0765','0766','0878','0971','1072','1194']
for seq in tqdm(fail_seqs):
    seq_path = join(datapath,seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    if meta['hand_flag'] == 'right':
        hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
        hand_trans = hand_params[:,:3]
        hand_theta = hand_params[:,3:51]
        mano_beta = hand_params[:,51:]
        active_obj = meta['active_obj']
        print(active_obj)
        obj_mesh_path = join(obj_path,active_obj,'simplified_scan_processed.obj')
        obj_mesh = trimesh.load(obj_mesh_path)
        obj_pose = torch.tensor(np.load(join(seq_path,active_obj+'_pose_trans.npy'))).float()
        obj_rot = obj_pose[:,:,:3]
        obj_trans = obj_pose[:,:,3]
        obj_sdf = SDF(obj_mesh.vertices,obj_mesh.faces)
        for i in range(len(hand_params)):
            thumb_flag = False
            finger_flag = False
            save_flag = False
            mano_output = manolayer(hand_theta[i].unsqueeze(0), mano_beta[i].unsqueeze(0))
            mano_verts = mano_output.verts[0] - mano_output.joints[0, 0] + hand_trans[i]
            mano_verts = convert_to_obj_frame(mano_verts.squeeze(),
                                                obj_rot[i],
                                                obj_trans[i])
            # hand_faces = manolayer.get_mano_closed_faces()
            hand_contact = obj_sdf(mano_verts)
            # print(hand_contact.shape)
            if get_close(hand_contact) > 0:
                for tip in tips:
                    tip_contact = hand_contact[verts_part_d == tip]
                    tip_close = get_close(tip_contact)
                    print(tip,tip_close)
                    if tip == 15:
                        if tip_close > 0:
                            thumb_flag = True
                    else:
                        if tip_close > 0 :
                            finger_flag = True
                if thumb_flag and finger_flag:
                    grasp_dict[seq] = i
                    # print(seq,i)
                    break
            else:
                if i == len(hand_params) - 1:
                    print(seq,"failed.")
        # break
    else:
        continue

In [44]:
print(grasp_dict)

{'0001': 14, '0003': 14, '0006': 14, '0007': 13, '0008': 16, '0013': 8, '0019': 10, '0020': 19, '0023': 11, '0024': 10, '0026': 7, '0028': 14, '0031': 15, '0032': 14, '0035': 19, '0037': 20, '0038': 14, '0040': 14, '0042': 18, '0044': 35, '0047': 12, '0048': 18, '0051': 10, '0052': 61, '0055': 18, '0056': 20, '0058': 22, '0061': 16, '0062': 34, '0065': 25, '0067': 34, '0068': 13, '0071': 25, '0072': 27, '0075': 47, '0076': 30, '0079': 44, '0080': 47, '0083': 38, '0084': 38, '0087': 38, '0088': 2, '0091': 43, '0092': 42, '0094': 29, '0096': 36, '0099': 19, '0101': 36, '0102': 3, '0103': 43, '0105': 11, '0106': 8, '0107': 8, '0109': 1, '0111': 62, '0112': 29, '0113': 10, '0114': 46, '0115': 38, '0118': 42, '0119': 33, '0145': 52, '0146': 3, '0147': 72, '0148': 7, '0149': 18, '0150': 9, '0155': 17, '0156': 11, '0160': 48, '0161': 59, '0162': 63, '0163': 43, '0164': 64, '0168': 42, '0169': 33, '0170': 46, '0171': 19, '0172': 75, '0176': 26, '0177': 43, '0178': 34, '0179': 14, '0181': 18, '

In [45]:
goals = np.array(list(grasp_dict.values()))
print(goals.max())

192


In [37]:
goals_seq = list(grasp_dict.keys())
print(goals_seq)

['0001', '0003', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0017', '0019', '0020', '0023', '0024', '0026', '0028', '0031', '0032', '0035', '0037', '0038', '0040', '0042', '0044', '0047', '0048', '0051', '0052', '0055', '0056', '0058', '0061', '0062', '0065', '0067', '0068', '0071', '0072', '0075', '0076', '0079', '0080', '0083', '0084', '0087', '0088', '0091', '0092', '0094', '0096', '0099', '0101', '0102', '0103', '0105', '0106', '0107', '0109', '0111', '0112', '0113', '0114', '0115', '0118', '0119', '0121', '0124', '0127', '0130', '0133', '0140', '0143', '0145', '0146', '0147', '0148', '0149', '0150', '0155', '0156', '0160', '0161', '0162', '0163', '0164', '0168', '0169', '0170', '0171', '0172', '0176', '0177', '0178', '0179', '0181', '0183', '0186', '0187', '0189', '0191', '0192', '0194', '0195', '0196', '0197', '0198', '0200', '0201', '0203', '0204', '0205', '0206', '0207', '0209', '0210', '0211', '0214', '0215', '0219', '0220', '0221',

In [40]:
seq_002 = []
for seq in goals_seq:
    seq_path = join(datapath,seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    if meta['hand_flag'] == 'right':
        hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
        hand_trans = hand_params[:,:3]
        hand_theta = hand_params[:,3:51]
        mano_beta = hand_params[:,51:]
        active_obj = meta['active_obj']
        # print(active_obj)
        if active_obj.startswith('002_qua') or active_obj.startswith('002_tri'):
            seq_002.append(seq)



In [42]:
# seq_002
for seq in seq_002:
    del grasp_dict[seq]

# 去掉失败的部分

In [56]:
for seq,index in grasp_dict.items():
    seq_path = join(datapath,seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    # print(meta)
    if meta['hand_flag'] == 'right':
        hand_params = np.load(join(seq_path,'mano/poses_right.npy'))
    #     meta['goal_index'] = index
    # with open(meta_path,'wb')as f:
    #     pickle.dump(meta, f)
    break

In [58]:
hand_pose = hand_params[:5,:3]

In [64]:
valid_seqs = list(grasp_dict.keys())

In [65]:
len(valid_seqs)

765

In [55]:
trainpath = '/root/code/seqs/gazehoi_list_train_new.txt'
testpath = '/root/code/seqs/gazehoi_list_test_new.txt'
        
with open(trainpath,'r') as f:
    train_list = f.readlines()
with open(testpath,'r') as f:
    test_list = f.readlines()

valid_seqs = []
for info in train_list:
    seq = info.strip()
    valid_seqs.append(seq)
for info in test_list:
    seq = info.strip()
    valid_seqs.append(seq)


In [56]:
len(valid_seqs)

765

In [71]:
len(seqs),len(new_seqs)

(750, 684)

In [72]:
f=open("/root/code/seqs/gazehoi_list_train_new.txt","w")
 
# f.writelines(test)
for line in new_seqs:
    f.write(line+'\n')
f.close()

In [78]:
path1 = '/root/code/OmniControl/dataset/kit_spatial_norm/Mean_raw.npy'
print(np.load(path1).shape)

(63,)


In [79]:
path1 = '/root/code/OmniControl/dataset/kit_mean.npy' 
print(np.load(path1).shape)

(251,)


# 计算mean

In [ ]:
valid_seqs

In [63]:
# seqs = sorted(os.listdir(valid_seqs))
from tqdm import *
hands_list = []
joint_list = []
for seq in tqdm(valid_seqs):
    seq_path = join('/root/code/seqs/1205_data/',seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    if meta['hand_flag'] == 'right':
        hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
        hand_trans = hand_params[:,:3]
        hand_rot = hand_params[:,3:6]
        hand_theta = hand_params[:,3:51]
        mano_beta = hand_params[:,51:]

        gt_output = manolayer(hand_theta, mano_beta)
        # 相对表示
        gt_joints = gt_output.joints - gt_output.joints[:, 0].unsqueeze(1) + hand_trans.unsqueeze(1)
        joint_list.append(gt_joints)
        # hands_list.append(hand_params)
        # hand_trans = hand_params[:,:3]
        # hand_theta = hand_params[:,3:51]
        # mano_beta = hand_params[:,51:]

100%|██████████| 765/765 [00:13<00:00, 55.88it/s] 


In [21]:
# seqs = sorted(os.listdir(valid_seqs))
from tqdm import *
from utils.data_util import obj_global2local_matrix, obj_matrix2rot6d
hands_list = []
joint_list = []
local_obj_pose_6d_list = []
global_obj_pose_6d_list = []
for seq in tqdm(valid_seqs):
    seq_path = join('/root/code/seqs/1205_data/',seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    if meta['hand_flag'] == 'right':
        obj_name_list = meta['obj_name_list']
        for obj in obj_name_list:
            obj_pose = torch.tensor(np.load(join(seq_path,obj+'_pose_trans.npy')).reshape(-1,3,4))
            obj_pose_local = obj_global2local_matrix(obj_pose.unsqueeze(0))
            obj_pose_local_6d = obj_matrix2rot6d(obj_pose_local)
            obj_pose_global_6d = obj_matrix2rot6d(obj_pose.unsqueeze(0))
            local_obj_pose_6d_list.append(obj_pose_local_6d.squeeze(0))
            global_obj_pose_6d_list.append(obj_pose_global_6d.squeeze(0))
    
        
        # hands_list.append(hand_params)
        # hand_trans = hand_params[:,:3]
        # hand_theta = hand_params[:,3:51]
        # mano_beta = hand_params[:,51:]

100%|██████████| 765/765 [00:03<00:00, 243.31it/s]


In [25]:
local_data = np.concatenate(local_obj_pose_6d_list, axis=0)
global_data = np.concatenate(global_obj_pose_6d_list, axis=0)

In [26]:
local_data.shape, global_data.shape

((474789, 9), (474789, 9))

In [32]:
# hands_pose = data[:,:51]
# print(hands_pose.shape)
data = global_data
mean = data.mean(axis=0)
std = data.std(axis=0)

std[:3] = std[:3].mean() / 1.0
std[3:] = std[3:].mean() / 1.0

print(mean,std)


[ 0.79933405 -0.72934205 -1.19342614 -0.21386048  0.00379753  0.17496449
  0.09812263 -0.05653823 -0.04264879] [0.13808932 0.13808932 0.13808932 0.56078171 0.56078171 0.56078171
 0.56078171 0.56078171 0.56078171]


In [33]:
np.save('/root/code/OmniControl/dataset/gazehoi_global_obj_mean.npy', mean)
np.save('/root/code/OmniControl/dataset/gazehoi_global_obj_std.npy', std)

In [68]:
mean,std.shape

(array([[ 0.65283066, -0.87845784, -1.3190148 ],
        [ 0.6552333 , -0.84926456, -1.3022171 ],
        [ 0.66498834, -0.83184993, -1.2834679 ],
        [ 0.68043625, -0.82334834, -1.267413  ],
        [ 0.69987905, -0.81139255, -1.2479547 ],
        [ 0.65505785, -0.84640986, -1.2444972 ],
        [ 0.67369163, -0.8385318 , -1.2257823 ],
        [ 0.6897996 , -0.83293426, -1.2187285 ],
        [ 0.7052852 , -0.82371163, -1.2139604 ],
        [ 0.663287  , -0.8641404 , -1.2366261 ],
        [ 0.68344283, -0.85599595, -1.2219536 ],
        [ 0.69989264, -0.8476628 , -1.2189465 ],
        [ 0.7173478 , -0.8390383 , -1.2199993 ],
        [ 0.6740651 , -0.88206345, -1.2463697 ],
        [ 0.68853736, -0.8710868 , -1.2313952 ],
        [ 0.7053893 , -0.86183923, -1.2284079 ],
        [ 0.7201399 , -0.8533746 , -1.2264625 ],
        [ 0.6846773 , -0.89362437, -1.2567991 ],
        [ 0.698029  , -0.88793355, -1.2473216 ],
        [ 0.70955783, -0.8794469 , -1.2435588 ],
        [ 0.7200982 

In [95]:
std[:3] = std[:3].mean() / 1.0
std[3:6] = std[3:6].mean() / 1.0
std[6:] = std[6:].mean() / 1.0

In [67]:
np.save('/root/code/OmniControl/dataset/gazehoi_global_joint_mean.npy', mean)
np.save('/root/code/OmniControl/dataset/gazehoi_global_joint_std.npy', std)

In [97]:
mean.shape

(51,)

In [28]:
res = np.load('/root/code/OmniControl/save/my_omnicontrol2/samples_my_omnicontrol2_000050000_seed10_predefined/results.npy',allow_pickle=True).item()

In [108]:
print(res.keys())

dict_keys(['motion', 'lengths', 'hint', 'seqs', 'num_samples', 'num_repetitions'])


In [113]:
len(set(res['seqs']))

81

In [29]:
def simple_eval(x, hint, n_joints=22):
    # mask = hint != 0
    # hint = hint * raw_std + raw_mean
    # hint = hint * mask
    # hint = hint.transpose((0, 2, 1))
    # print(hint.shape,motion.shape)
    # # hint = hint.reshape(hint.shape[0], , -1)
    # mask = hint.sum(axis=1, keepdims=True) != 0
    # print(mask.shape)
    # loss = np.linalg.norm((motion - hint) * mask, axis=1)
    # loss = loss.sum() / mask.sum()
    mask_hint = hint.view(hint.shape[0], hint.shape[1], 17, 3).sum(dim=-1, keepdim=True) != 0

    x_ = x.permute(0, 3, 2, 1).contiguous()
    x_ = x_.squeeze(2)
    bs,nf,_ = x_.shape
    print(hint.shape,x.shape,mask_hint.shape)
    loss = torch.norm((x_.reshape(bs,nf,-1,3) - hint.reshape(bs,nf,-1,3)) * mask_hint, dim=-1)
    return loss

In [30]:
# from utils.simple_eval import simple_eval
all_motions = torch.tensor(res['motion'])
all_hint = torch.tensor(res['hint'])
loss = simple_eval(all_motions, all_hint)
# print(loss)
print("Guide: ",torch.mean(loss[:,0]))

torch.Size([81, 60, 51]) torch.Size([81, 51, 1, 60]) torch.Size([81, 60, 17, 1])
Guide:  tensor(0.0207, dtype=torch.float64)


In [116]:
all_hint.shape

(81, 60, 51)

# 可视化生成结果

In [47]:
import numpy as np
import torch
import pickle
path = '/root/code/OmniControl/save/guide_delay2/samples_guide_delay2_000030000_seed10_predefined/results.npy'
res = np.load(path,allow_pickle=True).item()
pred_motion = torch.tensor(res['motion']).permute(0, 3, 2, 1).squeeze(2).contiguous()
hint = torch.tensor(res['hint'])
print(hint.shape)
seqs = res['seqs']
pred_motion.shape

torch.Size([81, 60, 51])


torch.Size([81, 60, 51])

In [ ]:
from pytorch3d.transforms import rotation_6d_to_matrix,axis_angle_to_matrix

res_save_path = path.replace('npy','txt')
print(res_save_path)
stage1_eval(pred_motion, hint, seqs, res_save_path)

In [2]:
def cal_goal_err(x, hint):
    """
    每个joints的平均偏差?
    """
    mask_hint = hint.view(hint.shape[0], hint.shape[1], 17, 3).sum(dim=-1, keepdim=True) != 0
    x_ = x
    print(x.shape)
    bs,nf,_ = x_.shape
    loss = torch.norm((x_.reshape(bs,nf,-1,3) - hint.reshape(bs,nf,-1,3)) * mask_hint, dim=-1)
    # print(loss[:,0])
    loss = loss[:,0] # bs,17
    # loss = torch.sum(loss) / bs
    loss = torch.mean(loss[:,0])
    return loss

def stage1_eval(pred_motion, all_hint, all_seqs, res_save_path):
    pred_motion = torch.tensor(pred_motion)
    hint = torch.tensor(all_hint)
    goal_err = cal_goal_err(pred_motion,hint)

    num_samples = pred_motion.shape[0]


    datapath = '/root/code/seqs/1205_data/'
    manolayer = ManoLayer(mano_assets_root='/root/code/CAMS/data/mano_assets/mano',side='right')
    obj_path = '/root/code/seqs/object/'

    total_hand_T_error = 0
    total_hand_R_error = 0
    total_mpjpe = 0 # root-relative
    total_traj_invalid_10 = 0
    total_traj_invalid_20 = 0
    total_traj_num = 0
    total_hand_R_error = 0

    for i in range(pred_motion.shape[0]):
        seq = seqs[i]
        seq_path = join(datapath,seq)

        meta_path = join(seq_path,'meta.pkl')
        with open(meta_path,'rb')as f:
            meta = pickle.load(f)
            
        active_obj = meta['active_obj']
        obj_pose = np.load(join(seq_path,active_obj+'_pose_trans.npy'))
        
        goal_index = meta['goal_index']
        obj_pose = torch.tensor(obj_pose[:goal_index+1]).float()

        if goal_index < 59:
            hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))[:goal_index+1]
        else:
            hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))[goal_index-59:goal_index+1]

        hand_trans = hand_params[:,:3]
        hand_rot = hand_params[:,3:6]
        hand_theta = hand_params[:,3:51]
        mano_beta = hand_params[:,51:]

        pred_trans = torch.flip(pred_motion[i,:goal_index+1,:3],dims=[0])
        pred_theta = torch.flip(pred_motion[i,:goal_index+1,3:],dims=[0])
        pred_rot = torch.flip(pred_motion[i,:goal_index+1,3:6],dims=[0])

        pred_output = manolayer(pred_theta, mano_beta)
        # 相对表示
        pred_joints = pred_output.joints - pred_output.joints[:, 0].unsqueeze(1)
        gt_output = manolayer(hand_theta, mano_beta)
        # 相对表示
        gt_joints = gt_output.joints - gt_output.joints[:, 0].unsqueeze(1)
        mpjpe = torch.sum(torch.norm(pred_joints - gt_joints, dim=-1)) / (hand_trans.shape[0] * 21)
        total_mpjpe += mpjpe

        hand_T_error = torch.sum(torch.norm(hand_trans-pred_trans,p=2,dim=-1)) / hand_trans.shape[0]
        total_hand_T_error += hand_T_error

        hand_rot = axis_angle_to_matrix(hand_rot)
        pred_rot = axis_angle_to_matrix(pred_rot)
        hand_rot = torch.einsum('...ij->...ji', [hand_rot])
        hand_R_error = (torch.einsum('fpn,fnk->fpk',hand_rot,pred_rot) - torch.eye(3).unsqueeze(0).repeat(hand_rot.shape[0],1,1)).reshape(-1,9) # nf,3,3
        hand_R_error =  torch.sum(torch.norm(hand_R_error,dim=-1)) /hand_trans.shape[0]
        total_hand_R_error += hand_R_error

        traj_error = torch.norm(hand_trans-pred_trans,p=2,dim=1) 
        traj_invalid_10 = torch.sum(traj_error > 0.1)
        traj_invalid_20 = torch.sum(traj_error > 0.2)
        total_traj_invalid_10 += traj_invalid_10
        total_traj_invalid_20 += traj_invalid_20
        total_traj_num += traj_error.shape[0]
    
    with open(res_save_path, 'w') as f:
        f.write(f'Goal Error:{goal_err:.6f}\n')
        f.write(f'Traj Error (<10cm):{(total_traj_invalid_10/total_traj_num):.6f}\n')
        f.write(f'Traj Error (<20cm):{(total_traj_invalid_20/total_traj_num):.6f}\n')
        f.write(f'Hand Trans Error:{total_hand_T_error/num_samples:.6f}\n')
        f.write(f'Hand Rot Error:{total_hand_R_error/num_samples:.6f}\n')
        f.write(f'MPJPE :{total_mpjpe/num_samples:.6f}\n')

In [6]:

from manotorch.manolayer import ManoLayer
from os.path import join
import json
import os
os.environ['MUJOCO_GL'] = 'osmesa'
os.environ['PYOPENGL_PLATFORM'] = 'osmesa'
def vis_smpl(out_path, image, nf, vertices, faces, camera_R, camera_T, K = np.array([1031.8450927734375, 0.0, 932.1596069335938, 0.0, 1022.4588623046875, 541.9437255859375, 0.0, 0.0, 1.0]).reshape((3,3))):
    outname = os.path.join(out_path, '{:06d}.jpg'.format(nf))
    # if os.path.exists(outname): return
    render_data = {}
    assert vertices.shape[1] == 3 and len(vertices.shape) == 2, 'shape {} != (N, 3)'.format(vertices.shape)
    render_data = {"vertices": vertices, "faces": faces, "vid":0, "name": "human_{}_0".format(nf)}

    camera = {"K": K,
        "R": camera_R,
        "T":camera_T}
    from visualize.renderer import Renderer
    render = Renderer(height=3840, width=2160, faces=None)
    image_vis, depth = render.render(render_data, camera, image, add_back=True)
    # print(depth)
    cv2.imwrite(outname, image_vis)
    return image_vis, depth  

datapath = '/root/code/seqs/1205_data/'
manolayer = ManoLayer(mano_assets_root='/root/code/CAMS/data/mano_assets/mano',side='right')
faces = manolayer.th_faces
view = 5
calib_path = "/root/code/seqs/calibration_all.json"
with open(calib_path) as f:
    calib_dome = json.load(f)
    f.close()
camera_pose = np.vstack((np.asarray(calib_dome[str(view)]['RT']).reshape((3,4)), np.ones(4) ))
K = np.asarray(calib_dome[str(view)]['K']).reshape((3,3))

for i in range(pred_motion.shape[0]):
    seq = seqs[i]
    seq_path = join(datapath,seq)
    hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
    hand_trans = hand_params[:,:3]
    hand_theta = hand_params[:,3:51]
    mano_beta = hand_params[:,51:]

    pred_trans = pred_motion[i,:,:3]
    pred_theta = pred_motion[i,:,3:]
    mano_output = manolayer(pred_theta, mano_beta[:goal+1])
    print(mano_output.verts.shape,mano_output.joints.shape,pred_trans.shape)
    mano_verts = mano_output.verts - mano_output.joints[:, 0].unsqueeze(1) + pred_trans.unsqueeze(1)
    render_out = join('/root/code/OmniControl/save/guide_delay2/samples_guide_delay2_000030000_seed10_predefined/','render')
    os.makedirs(render_out,exist_ok=True)
    for k in range(len(mano_verts)):
        img = np.zeros((2160, 3840,3), np.uint8) 
        
        _, depth = vis_smpl(render_out, img, k, mano_verts[k], faces,camera_pose[:3,:3],camera_pose[:3,3],K)
        break

NameError: name 'pred_motion' is not defined

In [77]:
a = np.random.randn(1,10)
b = a[:,-10:]
print(a)
print(b)

[[-0.90727966  0.46295225 -0.35474287  1.6237592  -0.29448975 -1.81806888
   0.26575419  1.02130978 -1.11818234  2.30116171]]
[[-0.90727966  0.46295225 -0.35474287  1.6237592  -0.29448975 -1.81806888
   0.26575419  1.02130978 -1.11818234  2.30116171]]


In [100]:
def lengths_to_mask(lengths, max_len):
    # max_len = max(lengths)
    mask = torch.arange(max_len-1,-1,-1).expand(len(lengths), max_len) < lengths.unsqueeze(1)
    return mask

In [99]:
torch.arange(5,0,-1)

tensor([5, 4, 3, 2, 1])

In [101]:
lens = torch.tensor([[2],[3]])
print(lens.shape)
lengths_to_mask(lens,3)

torch.Size([2, 1])


tensor([[[False,  True,  True],
         [False,  True,  True]],

        [[ True,  True,  True],
         [ True,  True,  True]]])

In [111]:
hand_params.shape

torch.Size([150, 61])

In [78]:
import numpy as np
from os.path import join
import torch
from pytorch3d.transforms import  quaternion_to_axis_angle,axis_angle_to_quaternion, axis_angle_to_matrix, matrix_to_rotation_6d,rotation_6d_to_matrix,matrix_to_axis_angle
from data_loaders.humanml.common.quaternion import qinv, qmul
seq_path = '/root/code/seqs/1205_data/0001/'
hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
hand_motion = hand_params[:,:51]
hand_motion = (hand_motion.unsqueeze(0).repeat(10,1,1)).contiguous()
print(hand_motion.shape)

torch.Size([10, 175, 51])


In [62]:
# from utils.data_util import  global2local_axis,local2global_axis

raw = hand_motion
# local = global2local_axis(raw)
# after_axis = local2global_axis(local)
print(raw.shape)
after_rot = axis2rot6d(raw)
print(after_rot.shape)
axis = rot6d2axis(after_rot)
print((after_axis-axis)[0,2])
print(torch.sum(after_axis-axis)/(1750*51))

torch.Size([10, 175, 51])
torch.Size([10, 175, 99])
tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  3.2167e+00, -3.4285e+00,
        -4.1685e+00, -7.4506e-09,  1.4901e-08, -5.9605e-08, -1.8626e-09,
         3.5482e-10, -5.9605e-08, -9.3132e-10,  8.7587e-11, -2.9802e-08,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  1.4901e-08, -1.9550e-09,
         0.0000e+00,  0.0000e+00,  2.3411e-10,  0.0000e+00, -2.9802e-08,
        -1.4901e-08,  0.0000e+00,  1.4901e-08,  0.0000e+00,  0.0000e+00,
         1.4901e-08, -2.3381e-10, -1.4901e-08,  7.4506e-09, -4.4703e-08,
        -5.9605e-08, -7.4506e-09,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  5.9605e-08, -5.9605e-08,  2.9802e-08,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00])
tensor(-0.0023)


In [76]:
theta = raw[0,2,3:].reshape(-1,3).unsqueeze(0)
# print(theta)
rot6d = matrix_to_rotation_6d(axis_angle_to_matrix(theta))
theta_after = matrix_to_axis_angle(rotation_6d_to_matrix(rot6d))
diff = theta_after - theta
# print(diff)
# print(abs(diff) > 1e-6)
print(axis_angle_to_quaternion(theta))
print(axis_angle_to_quaternion(theta_after))

tensor([[[ 5.3535e-01,  4.3242e-01, -4.6088e-01, -5.6036e-01],
         [ 9.5530e-01,  4.3895e-02, -9.3296e-02,  2.7707e-01],
         [ 9.8818e-01,  9.1567e-03,  0.0000e+00,  1.5302e-01],
         [ 9.9764e-01,  4.1005e-03,  0.0000e+00,  6.8522e-02],
         [ 9.6868e-01, -4.8997e-02,  2.3991e-02,  2.4225e-01],
         [ 9.6427e-01, -4.6077e-02,  0.0000e+00,  2.6090e-01],
         [ 9.9590e-01, -1.5728e-02, -3.7406e-09,  8.9057e-02],
         [ 9.5400e-01, -1.8771e-01,  1.1988e-01,  2.0069e-01],
         [ 9.9200e-01, -7.4046e-02,  0.0000e+00,  1.0220e-01],
         [ 9.9686e-01, -4.6456e-02,  7.4740e-09,  6.4119e-02],
         [ 9.7246e-01, -5.9505e-02,  1.2219e-01,  1.8935e-01],
         [ 9.5948e-01, -5.0303e-02,  0.0000e+00,  2.7725e-01],
         [ 9.9440e-01, -1.8873e-02,  0.0000e+00,  1.0402e-01],
         [ 9.0321e-01,  3.4466e-01, -1.8801e-01,  1.7339e-01],
         [ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.000

In [33]:
import torch
from pytorch3d.transforms import  quaternion_to_axis_angle,axis_angle_to_quaternion, axis_angle_to_matrix, matrix_to_rotation_6d,rotation_6d_to_matrix,matrix_to_axis_angle
from data_loaders.humanml.common.quaternion import qinv, qmul
def axis2rot6d(motion):
    # motion (bs, frames, 51)
    bs, nf, _ = motion.shape
    motion_rot_axis = motion[:,:,3:].reshape(bs,nf,-1,3)
    motion_rot_6d = matrix_to_rotation_6d(axis_angle_to_matrix(motion_rot_axis))
    motion_rot_6d = motion_rot_6d.reshape(bs, nf, -1)
    motion = torch.cat((motion[:,:,:3], motion_rot_6d),dim=-1)
    return motion

def rot6d2axis(motion):
    # motion (bs, frames, 99)
    bs, nf, _ = motion.shape
    motion_rot_6d = motion[:,:,3:].reshape(bs,nf,-1,6)
    motion_axis = matrix_to_axis_angle(rotation_6d_to_matrix(motion_rot_6d))
    motion_axis = motion_axis.reshape(bs, nf, -1)
    motion = torch.cat((motion[:,:,:3], motion_axis),dim=-1)
    return motion 
    
def get_vel_axis(motion):
    # motion (bs, frames, 51)
    trans = motion[:,:,:3]
    rot_axis = motion[:,:,3:6]
    l_velocity = trans[:,1:] - trans[:,:-1] #bs, frames-1 ,3

    rot_qua = axis_angle_to_quaternion(rot_axis)
    print(rot_qua[:,:-1].shape)
    r_velocity = qmul(rot_qua[:,1:], qinv(rot_qua[:,:-1]))
    r_velocity = quaternion_to_axis_angle(r_velocity)

    return l_velocity, r_velocity

def global2local_axis(motion):
    # 只对global RT做相对变换
    l_velocity, r_velocity = get_vel_axis(motion)
    motion[:,1:,:3] = l_velocity
    motion[:,1:,3:6] = r_velocity
    return motion


def local2global_axis(motion):
    # 只对global RT做相对变换
    l_velocity = motion[:,:,:3]
    r_velocity = motion[:,:,3:6]

    trans = torch.cumsum(l_velocity,dim=1)

    r_velocity = axis_angle_to_quaternion(r_velocity)
    nf = l_velocity.shape[1]
    rot = r_velocity[:,0].unsqueeze(1)
    cur_r = rot
    for i in range(1,nf):
        rot_i = qmul(r_velocity[:,i].unsqueeze(1), cur_r)
        cur_r = rot_i
        # print(rot_i.shape)
        rot = torch.cat((rot,rot_i),dim=1)
    rot = quaternion_to_axis_angle(rot)
    motion[:,:,:3] = trans
    motion[:,:,3:6] = rot

    return motion




In [104]:
def get_vel_axis_by_matrix(motion):
    trans = motion[:,:,:3]
    rot_axis = motion[:,:,3:6]
    l_velocity = trans[:,1:] - trans[:,:-1] #bs, frames-1 ,3

    rot_matrix = axis_angle_to_matrix(rot_axis) # bs. frames, 3, 3
    matrix_0 = rot_matrix[:,:-1] # 需要求逆
    matrix_1 = rot_matrix[:,1:]
    matrix_0_inv = torch.einsum('...ij->...ji', [matrix_0])
    r_velocity = torch.einsum('fipn,fink->fipk',matrix_1,matrix_0_inv)
    r_velocity = matrix_to_axis_angle(r_velocity)
    return l_velocity, r_velocity

def global2local_axis_by_matrix(motion):
    l_velocity, r_velocity = get_vel_axis_by_matrix(motion)
    # motion[:,1:,:3] = l_velocity
    # motion[:,1:,3:6] = r_velocity
    new_motion = motion.clone()
    new_motion[:, 1:, :3] = l_velocity
    new_motion[:, 1:, 3:6] = r_velocity
    return new_motion

def local2global_axis_by_matrix(motion):
     # 只对global RT做相对变换
    l_velocity = motion[:,:,:3]
    r_velocity = motion[:,:,3:6]

    trans = torch.cumsum(l_velocity,dim=1)

    r_velocity = axis_angle_to_matrix(r_velocity)
    print(r_velocity.shape)
    # matrices = [r for r in r_velocity]
    # rot = torch.chain_matmul(*matrices)
    nf = l_velocity.shape[1]
    rot = r_velocity[:,0].unsqueeze(1) #bs,1,3,3
    cur_r = rot
    for i in range(1,nf):
        # print(r_velocity[:,i].unsqueeze(1).shape, cur_r.shape)
        rot_i = torch.einsum('fipn,fink->fipk',r_velocity[:,i].unsqueeze(1),cur_r)
        cur_r = rot_i
        rot = torch.cat((rot,rot_i),dim=1)

    print(rot.shape)
    rot = matrix_to_axis_angle(rot)
    print(rot.shape)
    new_motion = motion.clone()
    new_motion[:,:,:3] = trans
    new_motion[:,:,3:6] = rot
    return new_motion

In [105]:
local = global2local_axis_by_matrix(hand_motion)
global_after = local2global_axis_by_matrix(local)

torch.Size([10, 175, 3, 3])
torch.Size([10, 175, 3, 3])
torch.Size([10, 175, 3])


In [106]:
local = global2local_axis(hand_motion)
global_after = local2global_axis(local)

torch.Size([10, 174, 4])


In [2]:
trainpath = '/root/code/seqs/gazehoi_list_train_new.txt'
testpath = '/root/code/seqs/gazehoi_list_test_new.txt'
        
with open(trainpath,'r') as f:
    train_list = f.readlines()
with open(testpath,'r') as f:
    test_list = f.readlines()

valid_seqs = []
for info in train_list:
    seq = info.strip()
    valid_seqs.append(seq)
for info in test_list:
    seq = info.strip()
    valid_seqs.append(seq)

In [2]:
len(valid_seqs)

765

In [18]:
import pickle
from sklearn.neighbors import KDTree
from tqdm import *
import torch
import numpy as np
from os.path import join
motion_list = []
hint_list = []
lens = []
obj_path = '/root/code/seqs/object/'

def convert_to_obj_frame(points, obj_pose):
    # points(frames,5,3)
    # obj_pose (frames,3,4)

    # pc = (obj_rot.T @ (pc - obj_trans).T).T
    obj_T = obj_pose[:,:3,3].unsqueeze(1)
    # print(obj_T.shape)
    points = points - obj_T
    # hand_rot = torch.einsum('...ij->...ji', [hand_rot])
    # hand_R_error = (torch.einsum('fpn,fnk->fpk',hand_rot,pred_rot) - torch.eye(3).unsqueeze(0).repeat(hand_rot.shape[0],1,1)).reshape(-1,9) # nf,3,3
       
    points = torch.einsum('...ij->...ji', [points])
    obj_R = obj_pose[:,:3,:3]
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    new_points = torch.einsum('fpn,fnk->fpk',obj_R,points)
    new_points = torch.einsum('...ij->...ji', [new_points])

    return new_points

for seq in tqdm(valid_seqs):
    seq_path = join('/root/code/seqs/1205_data/',seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    goal_index = meta['goal_index']
    active_obj = meta['active_obj']
    if meta['hand_flag'] == 'right':
        hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy'))).float()
        lens.append(hand_params.shape[0])
        """
        # obj_pose = torch.tensor(np.load(join(seq_path,active_obj+'_pose_trans.npy'))).float()
        # # print(hand_params.shape, obj_pose.shape)
        # obj_verts = torch.tensor(np.load(join(obj_path,active_obj,'resampled_500_trans.npy'))).float()
        # goal_obj_pose = obj_pose[goal_index]
        # # obj_verts = obj_verts @ goal_obj_pose[:3,:3].T + goal_obj_pose[:3,3].reshape(1,3)
        # kdt = KDTree(obj_verts)
        
        # hand_trans = hand_params[:,:3]
        # hand_rot = hand_params[:,3:6]
        # hand_theta = hand_params[:,3:51]
        # mano_beta = hand_params[:,51:]

        # gt_output = manolayer(hand_theta, mano_beta)
        # gt_joints = gt_output.joints - gt_output.joints[:, 0].unsqueeze(1) + hand_trans.unsqueeze(1)

        # tips = [15,3,6,12,9]
        # tips_T = gt_joints[goal_index:,tips]
        # # print(tips_T.shape)
        # obj_pose = obj_pose[goal_index:]
        # tips_to_obj = convert_to_obj_frame(tips_T, obj_pose) # (frames,5,3) (frames,3,4)
        # # print(tips_to_obj.shape)
        # ds, ids = kdt.query(tips_to_obj.reshape(-1,3), k=1)
        # ids = torch.from_numpy(ids)
        # ds = torch.from_numpy(ds)
        # # print(ds.shape, ids.shape)
        # target_verts = obj_verts[ids.squeeze()].reshape(-1,5,3)
        # ids = ids.reshape(-1,5)
        # # print(ids.shape, target_verts.shape)
        # # print(ids[0].shape)
        
        # target_ds = ds.reshape(-1,5)
        # # print(target_verts[0],target_ds[0],tips_to_obj[0])
        # # dis = torch.norm(target_verts - tips_to_obj,dim=-1)
        # # print(dis[0],target_ds[0])
        # # print(target_verts[0].shape,target_ds[0].shape)
        # res = torch.cat((ids[0].unsqueeze(-1),target_ds[0].unsqueeze(-1)),dim=-1)
        # # print(res.shape)
        # # break
        # # print(res)
        # # print(res.shape)

        # # break
        # np.save(join(seq_path,'tips_closest_id_and_dis.npy'), res)
        """

        # pc = tips_T[0]
        # obj_rot = goal_obj_pose[:3,:3]
        # obj_trans = goal_obj_pose[:3,3]
        # pc = (obj_rot.T @ (pc - obj_trans).T).T
        # nn = torch.from_numpy(kdt.query(pc.reshape(-1,3), k=1)[1])
        # target_verts = obj_verts[nn.squeeze()]
        # print(target_verts)
        # print(pc)



        # break
       


        # obj_T = obj_T.unsqueeze(1)
        # tip_obj_dis = torch.norm(tips_T - obj_T,dim=-1).numpy()
        # np.save(join(seq_path,'tip_distance.npy'), tip_obj_dis)

        # hand_T = hand_params[goal_index:,:3]
        # obj_T = obj_pose[goal_index:,:3,3]
        # distance = torch.norm(hand_T-obj_T,dim=-1).numpy()
        # np.save(join(seq_path,'global_distance.npy'), distance)
        
        # print(distanc
        # seq_len = hand_params.shape[0] - goal_index
        # lens.append(seq_len)


        # print(tip_obj_dis.shape, tip_obj_dis[0])

        # break
        # joint_list.append(gt_joints)
        # hand_motion = hand_params[:,:51]

        # local_motion = global2local_axis_by_matrix(hand_motion.unsqueeze(0))
        # local_motion_6d = axis2rot6d(local_motion)
        # motion_list.append(local_motion_6d.squeeze(0))

        # global_motion_6d = axis2rot6d(hand_motion.unsqueeze(0))
        # hint_list.append(global_motion_6d.squeeze(0))


  0%|          | 0/765 [00:00<?, ?it/s]

100%|██████████| 765/765 [00:00<00:00, 3248.11it/s]


In [19]:
lens = np.array(lens)
lens.max(),lens.min()

(345, 84)

In [129]:
# print(hands_list[0].shape)
local_motions = torch.cat(motion_list, dim=0).numpy()
global_motions = torch.cat(hint_list, dim=0).numpy()
# print(hand_motions.shape)

In [133]:
mean = global_motions.mean(axis=0)
std = global_motions.std(axis=0)
std[:3] = std[:3].mean() / 1.0
std[3:6] = std[3:6].mean() / 1.0
std[6:] = std[6:].mean() / 1.0

In [134]:
np.save('/root/code/OmniControl/dataset/gazehoi_global_motion_6d_mean.npy', mean)
np.save('/root/code/OmniControl/dataset/gazehoi_global_motion_6d_std.npy', std)

In [136]:
R0 = torch.eye(3)
print(R0)
matrix_to_axis_angle(R0)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


tensor([0., 0., 0.])

In [4]:
manolayer = ManoLayer(mano_assets_root='/root/code/CAMS/data/mano_assets/mano',side='right',cuda=True)

# 通过物体轨迹反推手轨迹

In [41]:
from tqdm import *
from os.path import join
import pickle
import torch
import numpy as np
import trimesh

def get_trans_obj_verts(obj_verts,obj_pose):
    """
    obj_verts: (N,3) -- T,N
    obj_pose: (T,3,4)
    """
    nf = obj_pose.shape[0]
    N = obj_verts.shape[0]
    # obj_pose = obj_pose.unsqueeze(1).repeat(1,N,1,1)
    obj_R = obj_pose[:,:3,:3]
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    obj_T = obj_pose[:,:3,3].unsqueeze(1)
    
    obj_verts = obj_verts.unsqueeze(0).repeat(nf,1,1)
    trans_obj_verts = torch.einsum('fpn,fnk->fpk',obj_verts,obj_R) + obj_T
    return trans_obj_verts


for seq in tqdm(valid_seqs):
    seq_path = join('/root/code/seqs/1205_data/',seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    goal_index = meta['goal_index']
    active_obj = meta['active_obj']
    if meta['hand_flag'] == 'right':
        hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy'))).float()
        obj_pose = torch.tensor(np.load(join(seq_path,active_obj+'_pose_trans.npy'))).float()
        obj_R = obj_pose[:,:3,:3]
        # print(hand_params.shape, obj_pose.shape)
        # obj_verts = torch.tensor(np.load(join(obj_path,active_obj,'resampled_500_trans.npy'))).float()
        obj_verts = torch.tensor(trimesh.load(join(obj_path,active_obj,'simplified_scan_processed.obj')).vertices).float()
        goal_obj_pose = obj_pose[goal_index]
        # obj_verts = obj_verts @ goal_obj_pose[:3,:3].T + goal_obj_pose[:3,3].reshape(1,3)
        # kdt = KDTree(obj_verts)
        
        hand_trans = hand_params[:,:3]
        hand_rot = hand_params[:,3:6]
        hand_theta = hand_params[:,3:51]
        mano_beta = hand_params[:,51:]

        goal_hand_trans = hand_trans[goal_index].unsqueeze(0)
        trans_obj_verts = get_trans_obj_verts(obj_verts,obj_pose)
        goal_obj_verts = trans_obj_verts[goal_index]
        dis = torch.cdist(goal_hand_trans,goal_obj_verts)
        min_dis,obj_idx = dis.min(dim=-1)

        contact_normal = goal_hand_trans - goal_obj_verts[obj_idx]
        reference_obj_rot = obj_R[goal_index]
        update_hand_T = torch.zeros_like(hand_trans[goal_index:])
        update_hand_R = torch.zeros_like(hand_rot[goal_index:])
        obj_R = obj_R[goal_index:]
        trans_obj_verts = trans_obj_verts[goal_index:]
        for t_idx in range(len(hand_params)-goal_index):
            relative_rot_mat = torch.matmul(obj_R[t_idx], reference_obj_rot.inverse())
            curr_contact_normal = torch.matmul(relative_rot_mat, contact_normal.T).squeeze(-1)
            update_hand_T[t_idx] = trans_obj_verts[t_idx, obj_idx] + curr_contact_normal
        # print(update_hand_T.shape)
        update_hand_T = update_hand_T.numpy()
        np.save(join(seq_path,'contact_hand_T_from_obj_T.npy'), update_hand_T)
        # print(hand_trans[goal_index:])
        # print(hand_trans[goal_index:] - update_hand_T)
        
        # break

  0%|          | 0/765 [00:00<?, ?it/s]

100%|██████████| 765/765 [00:47<00:00, 16.25it/s]


In [ ]:
# OMOMO

def process_hand_foot_contact_jpos(self, hand_foot_jpos, object_mesh_verts, obj_rot):
    # hand_foot_jpos: T X 1 X 3 
    # object_mesh_verts: T X Nv X 3 
    # obj_rot: T X 3 X 3 
    all_contact_labels = []
    all_object_c_idx_list = []
    all_dist = []

    obj_rot = torch.from_numpy(obj_rot).to(hand_foot_jpos.device)
    object_mesh_verts = object_mesh_verts.to(hand_foot_jpos.device)

    num_joints = hand_foot_jpos.shape[1]
    num_steps = hand_foot_jpos.shape[0]

    threshold = 0.03 # Use palm position, should be smaller. 
    
    joint2object_dist = torch.cdist(hand_foot_jpos, object_mesh_verts.to(hand_foot_jpos.device)) # T X 2 X Nv 
    
    all_dist, all_object_c_idx_list = joint2object_dist.min(dim=2) # T X 2  最小距离，物体上最近点的id
    all_contact_labels = all_dist < threshold # T X 2

    new_hand_foot_jpos = hand_foot_jpos.clone() # T X 2 X 3 

    # For each joint, scan the sequence, if contact is true, then use the corresponding object idx for the 
    # rest of subsequence in contact. 
    for j_idx in range(num_joints):
        continue_prev_contact = False 
        for t_idx in range(num_steps):
            if continue_prev_contact:
                relative_rot_mat = torch.matmul(obj_rot[t_idx], reference_obj_rot.inverse())
                curr_contact_normal = torch.matmul(relative_rot_mat, contact_normal[:, None]).squeeze(-1)

                new_hand_foot_jpos[t_idx, j_idx] = object_mesh_verts[t_idx, subseq_contact_v_id] + \
                    curr_contact_normal  # 3  
            
            elif all_contact_labels[t_idx, j_idx] and not continue_prev_contact: # The first contact frame 
                subseq_contact_v_id = all_object_c_idx_list[t_idx, j_idx] # 物体上点的id
                subseq_contact_pos = object_mesh_verts[t_idx, subseq_contact_v_id] # 3 开始接触这一帧，物体上和手在最近点的坐标

                contact_normal = new_hand_foot_jpos[t_idx, j_idx] - subseq_contact_pos # Keep using this in the following frames. 手-物体

                reference_obj_rot = obj_rot[t_idx] # 3 X 3 

                continue_prev_contact = True 

    return new_hand_foot_jpos 

In [17]:
import torch
import torch.optim as optim

# 假设您有一个包含一些参数的张量或变量
# 这里假设参数是一个大小为(10, 10)的张量，我们将对其进行切片优化
params = torch.randn(10, 10, requires_grad=True)

# 定义要优化的切片
sliced_params_to_optimize = params[:, :5]  # 例如，我们只优化第一列到第五列的切片

# 定义优化器，将切片参数添加到优化器中
optimizer = optim.SGD([sliced_params_to_optimize], lr=0.01)  # 选择合适的优化算法和学习率

# 在训练循环中，使用优化器来更新参数
for epoch in range(num_epochs):
    optimizer.zero_grad()  # 清除之前的梯度
    # 计算损失，这里假设您有一个损失函数
    loss = compute_loss(sliced_params_to_optimize)
    loss.backward()  # 计算梯度
    optimizer.step()  # 更新参数


ValueError: can't optimize a non-leaf Tensor

## 检查obj 表示方法转换

In [72]:
from utils.data_util import  obj_matrix2rot6d
from pytorch3d.transforms import  quaternion_to_axis_angle,axis_angle_to_quaternion, axis_angle_to_matrix, matrix_to_rotation_6d,rotation_6d_to_matrix,matrix_to_axis_angle
def obj_get_vel_matrix(motion):
    # bs, num_frames,3,4
    # 相对最后一帧的表示
    num_frames = motion.shape[1]
    # motion = motion.reshape(-1,num_frames,3,4)
    trans = motion[:,:,:3,3]
    rot_matrix = motion[:,:,:3,:3]
    l_velocity =  trans[:,:-1] - trans[:,1:] #bs, frames-1 ,3

    matrix_1 = rot_matrix[:,:-1] # 需要求逆
    matrix_0 = rot_matrix[:,1:]
    matrix_0_inv = torch.einsum('...ij->...ji', [matrix_0])
    r_velocity = torch.einsum('fipn,fink->fipk',matrix_1,matrix_0_inv)
    # r_velocity = matrix_to_rotation_6d(r_velocity)
    return l_velocity, r_velocity
def obj_global2local_matrix(motion):
    l_velocity, r_velocity = obj_get_vel_matrix(motion)
    # motion[:,1:,:3] = l_velocity
    # motion[:,1:,3:6] = r_velocity
    new_motion = motion.clone()
    new_motion[:, :-1, :3,3] = l_velocity
    new_motion[:, :-1, :3,:3] = r_velocity
    return new_motion
def obj_local2global_rot6d_by_matrix(motion):
     # 只对global RT做相对变换
    #  反过来 最后一帧是参考
    l_velocity = torch.flip(motion[:,:,:3],dims=[1])
    r_velocity = motion[:,:,3:9]

    trans = torch.flip(torch.cumsum(l_velocity,dim=1),dims=[1])

    r_velocity = rotation_6d_to_matrix(r_velocity)
    print(r_velocity.shape)
    nf = l_velocity.shape[1]
    rot = r_velocity[:,-1].unsqueeze(1) #bs,1,3,3
    cur_r = rot
    for i in range(nf-2,-1,-1):
        # print(i)
        rot_i = torch.einsum('fipn,fink->fipk',r_velocity[:,i].unsqueeze(1),cur_r)
        cur_r = rot_i
        rot = torch.cat((rot_i,rot),dim=1)

    rot = matrix_to_rotation_6d(rot)
    new_motion = motion.clone()
    new_motion[:,:,:3] = trans
    new_motion[:,:,3:9] = rot
    return new_motion

def obj_local2global_matrix(motion):
     # 只对global RT做相对变换
    #  反过来 最后一帧是参考
    
    bs,nf ,_= motion.shape
    new_motion = torch.zeros((bs,nf,4,3,4))
    motion = motion.reshape(bs,nf,4,9)
    new_motion = motion.clone()
    for k in range(4):
        l_velocity = torch.flip(motion[:,:,k,:3],dims=[1])
        r_velocity = motion[:,:,k,3:9]

        trans = torch.flip(torch.cumsum(l_velocity,dim=1),dims=[1])

        r_velocity = rotation_6d_to_matrix(r_velocity)
        nf = l_velocity.shape[1]
        rot = r_velocity[:,-1].unsqueeze(1) #bs,1,3,3
        cur_r = rot
        for i in range(nf-2,-1,-1):
            # print(i)
            rot_i = torch.einsum('fipn,fink->fipk',r_velocity[:,i].unsqueeze(1),cur_r)
            cur_r = rot_i
            rot = torch.cat((rot_i,rot),dim=1)

        # rot = matrix_to_rotation_6d(rot)
        
        new_motion[:,:,k,:3,3] = trans
        new_motion[:,:,k,:3,:3] = rot
    # new_motion = new_motion.reshape(bs,nf,-1)
    return new_motion
obj_pose = np.load('/root/code/seqs/1205_data/0001/001_book_4_pose.npy')
bs = obj_pose.shape[0]
nf = obj_pose.shape[1]
obj_pose = torch.tensor(obj_pose).unsqueeze(1).repeat(1,4,1,1).reshape()
print(obj_pose.shape)
obj_pose_local = obj_global2local_matrix(obj_pose.unsqueeze(0))
obj_pose_local_6d = obj_matrix2rot6d(obj_pose_local)
obj_pose_global_6d = obj_matrix2rot6d(obj_pose.unsqueeze(0)).squeeze(0)

new_global_matrix = obj_local2global_matrix(obj_pose_local_6d).squeeze(0)

torch.Size([175, 4, 3, 4])


IndexError: index 3 is out of bounds for dimension 3 with size 3

In [64]:
print(new_global_6d.shape)
old = rotation_6d_to_matrix(obj_pose_global_6d[:,3:])
new = rotation_6d_to_matrix(new_global_6d[:,3:])
print(old.shape, new.shape)
matrix_0_inv = torch.einsum('...ij->...ji', [old])
r_velocity = torch.einsum('ipn,ink->ipk',new,matrix_0_inv)
print(r_velocity)

torch.Size([150, 9])
torch.Size([150, 3, 3]) torch.Size([150, 3, 3])
tensor([[[ 1.0000e+00,  7.7716e-15,  2.5369e-14],
         [-7.7993e-15,  1.0000e+00, -4.6629e-14],
         [-2.5313e-14,  4.6602e-14,  1.0000e+00]],

        [[ 1.0000e+00,  7.4662e-15,  2.4869e-14],
         [-7.4940e-15,  1.0000e+00, -4.6324e-14],
         [-2.4925e-14,  4.6296e-14,  1.0000e+00]],

        [[ 1.0000e+00,  7.6605e-15,  2.5036e-14],
         [-7.6883e-15,  1.0000e+00, -4.5908e-14],
         [-2.5091e-14,  4.5935e-14,  1.0000e+00]],

        ...,

        [[ 1.0000e+00,  2.7756e-17,  4.4409e-16],
         [-8.3267e-17,  1.0000e+00, -5.2736e-16],
         [-3.8858e-16,  5.5511e-16,  1.0000e+00]],

        [[ 1.0000e+00,  1.1102e-16,  2.7756e-16],
         [-8.3267e-17,  1.0000e+00, -5.5511e-17],
         [-2.2204e-16,  8.3267e-17,  1.0000e+00]],

        [[ 1.0000e+00,  8.3267e-17, -5.5511e-17],
         [-5.5511e-17,  1.0000e+00,  5.5511e-17],
         [ 5.5511e-17,  0.0000e+00,  1.0000e+00]]], dtype